In [43]:
#libraries
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans

Solving environment: done

# All requested packages already installed.



In [44]:
#data
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page,'lxml')

In [45]:
#extracting table
wiki_table=soup.find('table')

column_names = ['Postal Code','Borough','Neighborhood']
df=pd.DataFrame(columns=column_names)

for row in wiki_table.findAll('tr'):
    data=[]
    for cell in row.find_all('td'):
        data.append(cell.text.strip())
    if len(data)==3:
        df.loc[len(df)] = data

#removing,grouping and reorganizing data        
df=df[df.Borough !='Not assigned']
df=df.groupby(['Postal Code', 'Neighborhood'], sort=False).agg(', '.join)
df.reset_index(inplace=True)
df=df.reindex(columns=['Postal Code','Borough','Neighborhood'])

df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [46]:
df.shape

(103, 3)

In [47]:
#read csv file
coordinates=pd.read_csv('http://cocl.us/Geospatial_data')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [48]:
#merged 2 tables
result = pd.merge(df,coordinates, on='Postal Code')
result.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [49]:
#filtered borough to downtown toronto
toronto_data = result[result['Borough'].str.contains('Toronto',regex=False)]
toronto_data.reset_index(inplace=True)
toronto_data.head()

,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [50]:
geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [51]:
#clustering
k=5
toronto_grouped_clustering = toronto_data.drop(['Postal Code','Borough','Neighborhood'], 1)
kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_
toronto_data.insert(0, 'Cluster Labels', kmeans.labels_)

In [52]:
toronto_data.head()

,Cluster Labels,index,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,3,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,3,4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,3,9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,3,15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,1,19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [53]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood'], toronto_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters